In [40]:
# !pip install langchain langchain_community
# !pip install sentence-transformers
# !pip install pypdf
# !pip install tiktoken

## Maestría en Inteligencia Artificial Aplicada (MNA)
### Proyecto Integrador
### Dra. Grettel Barceló Alonso / Dr. Carlos Alberto Villaseñor Padilla
### Avance 4. Implementación de Arquitectura RAG

### Integrantes
- A01794457 - Iossif Moises Palli Laura
- A01793984 - Brenda Zurazy Rodríguez Pérez
- A01794630 - Jesús Ramseths Echeverría Rivera

In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from transformers import TrainingArguments, AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, pipeline
import torch
import re

In [2]:
from huggingface_hub import login
# Autenticar con Hugging Face utilizando un token personal
login('')

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### Carga de los documentos

In [4]:
# Cargar un documento PDF y extraer sus páginas
loader = PyPDFLoader('./Ley de Fondos de Inversión.pdf', extract_images=False)
pages = loader.load()
# Dividir el texto en fragmentos más pequeños
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size = 256)
docs = text_splitter.split_documents(pages)

In [5]:
# Limpiar el texto de los documentos
for doc in docs:
    # Se elimina espacios y saltos de línea al inicio y al final
    text = doc.page_content.strip()
    text = text.replace('\n', ' ')

    # Se wliminan espacios duplicados
    text = re.sub(r'[ \t]+', ' ', text)

    # Actualizar
    doc.page_content = text

# Filtrar documentos vacíos
docs = [doc for doc in docs if doc.page_content.strip() != '']

In [6]:
# Imprimir el contenido de la primera página procesada
print(docs[0].page_content)

Para uso interno Para uso interno (3) LEY DE FONDOS DE INVERSIÓN Publicada en el Diario Oficial de la Federación el 4 de junio de 2001. Actualizada con las modificaciones publicadas en el propio Diario el 28 de junio de 2007, 10 d e enero y 13 de junio de 2014 , 20 de mayo de 2021 , 28 de diciembre de 2023 y 24 de enero de 2024.


### Creación de Embeddings

In [25]:
# Cargar un modelo de embeddings de Hugging Face para transformar el texto
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/multi-qa-MiniLM-L6-cos-v1")

# Definir la ruta donde se guardarán los embeddings
persist_path = "../embeddings_db_rag"
# Crear un almacén vectorial con los documentos y sus embeddings
vector_store = SKLearnVectorStore.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_path=persist_path,
    serializer="parquet"
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [26]:
# Fuerza a guardar los embeddings en la DB
vector_store.persist()

In [27]:
# Crear el "retriever", que es el mecanismo que buscará entre los embeddings
retriever = vector_store.as_retriever()

### Model Load

In [28]:
# Cargar un modelo de lenguaje preentrenado (Llama) para generación de texto
model_name = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(
    model_name,
    device_map='auto'
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [29]:
# Definir pipeline para la generación de texto
generate_text = pipeline(
    task='text-generation',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15,
)

In [49]:
# Configurar el modelo de lenguaje dentro del pipeline de Hugging Face
llm = HuggingFacePipeline(pipeline=generate_text)

In [31]:
prompt_template = """
Utiliza la siguiente información para responder la pregunta al final de manera concisa y precisa.

Información:
{context}

Pregunta: {question}

Respuesta:
"""

In [32]:
prompt = PromptTemplate(
    input_variables=["context","question"],
    template=prompt_template
)

In [50]:
# Crear la cadena de preguntas y respuestas (QA) basada en el sistema RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": prompt,
    }
)

query = "¿En cuál fecha fue publicada la Ley de Fondos de Inversión?"
result = qa_chain(query)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [51]:
output = result['result']

if "Respuesta:" in output:
    answer = output.split("Respuesta:")[-1].strip()
else:
    answer = output.strip()

print("Respuesta:")
print(answer)

Respuesta:
La Ley de Fondos de Inversión fue publicada el 4 de junio de 2001.


### Conclusión

El uso de Retrieval-Augmented Generation (RAG) proporciona una poderosa combinación de recuperación de información y generación de texto. En este caso, el sistema permite recuperar información relevante de un documento de manera efectiva y generar respuestas precisas y coherentes. 

El modelo RAG ofrece una ventaja clave al integrar un retriever que busca el contexto adecuado en una base de datos vectorial de documentos y luego utiliza un modelo generador para crear respuestas basadas en ese contexto.

Esto es especialmente útil en situaciones donde la información se distribuye en múltiples fuentes o documentos y se necesita una respuesta concisa.